In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Define constants
IMG_WIDTH, IMG_HEIGHT = 299, 299
BATCH_SIZE = 32
NUM_EPOCHS = 10
TRAIN_DIR = 'path/to/train/directory'
VAL_DIR = 'path/to/validation/directory'

# Define InceptionV3 model with pre-trained weights
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/dataset1",
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    "/content/drive/MyDrive/dataset2",
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# Use early stopping to prevent overfitting
early_stop = EarlyStopping(patience=3, verbose=1)

# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=NUM_EPOCHS,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    callbacks=[early_stop]
)

# Save trained model
model.save('signature_verification_model.h5')


87910968/87910968 [==============================] - 1s 0us/step
Found 120 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Epoch 1/10
3/3 [==============================] - 87s 36s/step - loss: 0.9068 - accuracy: 0.5417 - val_loss: 1.4043 - val_accuracy: 0.5104
Epoch 2/10
3/3 [==============================] - 67s 28s/step - loss: 0.8990 - accuracy: 0.5568 - val_loss: 0.6798 - val_accuracy: 0.5833
Epoch 3/10
3/3 [==============================] - 50s 21s/step - loss: 0.7564 - accuracy: 0.5795 - val_loss: 0.6614 - val_accuracy: 0.6458
Epoch 4/10
3/3 [==============================] - 65s 28s/step - loss: 0.6166 - accuracy: 0.6477 - val_loss: 0.6805 - val_accuracy: 0.6354
Epoch 5/10
3/3 [==============================] - 69s 29s/step - loss: 0.6596 - accuracy: 0.6875 - val_loss: 0.8300 - val_accuracy: 0.5208
Epoch 6/10
3/3 [==============================] - 67s 28s/step - loss: 0.6326 - accuracy: 0.6250 - val_loss: 0.6213 - val_accuracy: 0.6458
Epoch 7/10
3/3 [===

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load trained model
model = load_model('signature_verification_model.h5')

# Define constants
IMG_WIDTH, IMG_HEIGHT = 299, 299

# Load test image and preprocess it
test_image = image.load_img('/content/drive/MyDrive/dataset2/real/00102001.png', target_size=(IMG_WIDTH, IMG_HEIGHT))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image/255.0

# Predict class probability
class_prob = model.predict(test_image)

# Print prediction
if class_prob > 0.5:
    print('The signature is genuine with probability', class_prob[0][0])
else:
    print('The signature is forged with probability', 1 - class_prob[0][0])


1/1 [==============================] - 2s 2s/step
The signature is forged with probability 0.5729008316993713
